In [1]:
import h5py
import torch
import os
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
from torch.utils.data.dataset import Dataset
import numpy as np
import setting
import time
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import ComFunction as cf
import math

In [2]:
# import torch.backends.cudnn as cudnn
# from scipy.io import loadmat
# from scipy.io import savemat
# from sklearn.metrics import confusion_matrix
from vit_pytorch import ViT

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type=='cuda':
#     dtype = torch.float32
#     torch.set_default_tensor_type(torch.cuda.FloatTensor)
    dtype = torch.float64
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)
else:
#     dtype = torch.float32
#     torch.set_default_tensor_type(torch.FloatTensor)
    dtype = torch.float64
    torch.set_default_tensor_type(torch.DoubleTensor)
print(device.type)

cuda


In [4]:
class HyperData(Dataset):
    def __init__(self, data, labels, transfor):
        self.data = data
        self.transformer = transfor
        self.labels = labels
        

    def __getitem__(self, index):
        img = self.data[index,:,:]
        label = self.labels[index]
        return img, label

    def __len__(self):
        return len(self.labels)

    def __labels__(self):
        return self.labels

In [5]:
def gain_neighborhood_band(x_train, band, band_patch, patch=5):
    nn = band_patch // 2
    pp = (patch*patch) // 2
    x_train_reshape = x_train.reshape(x_train.shape[0], patch*patch, band)
    x_train_band = np.zeros((x_train.shape[0], patch*patch*band_patch, band),dtype=float)
    # 中心区域
    x_train_band[:,nn*patch*patch:(nn+1)*patch*patch,:] = x_train_reshape
    #左边镜像
    for i in range(nn):
        if pp > 0:
            x_train_band[:,i*patch*patch:(i+1)*patch*patch,:i+1] = x_train_reshape[:,:,band-i-1:]
            x_train_band[:,i*patch*patch:(i+1)*patch*patch,i+1:] = x_train_reshape[:,:,:band-i-1]
        else:
            x_train_band[:,i:(i+1),:(nn-i)] = x_train_reshape[:,0:1,(band-nn+i):]
            x_train_band[:,i:(i+1),(nn-i):] = x_train_reshape[:,0:1,:(band-nn+i)]
    #右边镜像
    for i in range(nn):
        if pp > 0:
            x_train_band[:,(nn+i+1)*patch*patch:(nn+i+2)*patch*patch,:band-i-1] = x_train_reshape[:,:,i+1:]
            x_train_band[:,(nn+i+1)*patch*patch:(nn+i+2)*patch*patch,band-i-1:] = x_train_reshape[:,:,:i+1]
        else:
            x_train_band[:,(nn+1+i):(nn+2+i),(band-i-1):] = x_train_reshape[:,0:1,:(i+1)]
            x_train_band[:,(nn+1+i):(nn+2+i),:(band-i-1)] = x_train_reshape[:,0:1,(i+1):]
    return x_train_band

In [6]:
EPOCH=setting.EPOCH
BATCH_SIZE=setting.BATCH_SIZE
LR=setting.LR

In [7]:
train_filename =setting.train_data_name +'_'+ str(setting.PATCH_SIZE) + '_'+ str(setting.DTYPE) +'.h5'
with h5py.File(train_filename,'r') as readfile:
    train=readfile['train_patch'][:]
    train_labels=readfile['train_labels'][:]
print('train size:', train.shape)
print('train label name:',np.unique(train_labels))
print(np.max(train))
print(np.min(train))

train_max=np.max(np.abs(train))
print('train max:',train_max)


train size: (2000, 5, 5, 204)
train label name: [0 1]
0.9285714030265808
0.04900568351149559
train max: 0.9285714030265808


In [8]:
train_band = gain_neighborhood_band(train, setting.band, setting.band_patches, setting.PATCH_SIZE)
print(train_band.shape)
train_band =train_band.transpose(0,2,1)
print(train_band .shape)

(2000, 25, 204)
(2000, 204, 25)


In [9]:
val_filename = setting.val_data_name +'_'+ str(setting.PATCH_SIZE) +'_'+ str(setting.DTYPE) +'.h5'
with h5py.File(val_filename,'r') as readfile:
    val=readfile['val_patch'][:]
    val_labels=readfile['val_labels'][:]
print('val size:', val.shape)
print('val label name:',np.unique(val_labels))
print(np.max(val))
print(np.min(val))

val_max=np.max(np.abs(val))
print('val max:',val_max)


val_band = gain_neighborhood_band(val, setting.band, setting.band_patches, setting.PATCH_SIZE)
print(val_band.shape)
val_band =val_band.transpose(0,2,1)
print(val_band.shape)

val size: (5000, 5, 5, 204)
val label name: [0 1]
0.9399999976158142
0.05027933046221733
val max: 0.9399999976158142
(5000, 25, 204)
(5000, 204, 25)


In [10]:
# cpu
# train_set=HyperData(train_band,train_labels, None)
# trainloader= Data.DataLoader(dataset=train_set,batch_size=BATCH_SIZE,shuffle=True, num_workers=0)

# val_set=HyperData(val_band, val_labels, None)
# valloader= Data.DataLoader(dataset=val_set,batch_size=BATCH_SIZE,shuffle=False, num_workers=0)

In [11]:
# # gpu
train_set=HyperData(train_band,train_labels, None)
trainloader= Data.DataLoader(dataset=train_set,batch_size=BATCH_SIZE,shuffle=True, num_workers=0,generator=torch.Generator(device='cuda'))

val_set=HyperData(val_band, val_labels, None)
valloader= Data.DataLoader(dataset=val_set,batch_size=BATCH_SIZE,shuffle=False, num_workers=0)

In [12]:
time_str=time.strftime("%Y_%m_%d_%H_%M", time.localtime())

writer_name=os.path.join(setting.writer_name,'nn_'+time_str)
print(writer_name)
writer = SummaryWriter(writer_name)
# tensorboard --logdir=runs

runs/nn_2022_10_31_17_16


In [13]:
net=ViT(
    image_size = setting.PATCH_SIZE,
    near_band = setting.band_patches,
    num_patches = setting.band,
    num_classes = setting.num_class,
    dim = 64,
    depth = 5,
    heads = 4,
    mlp_dim = 8,
    dropout = 0.1,
    emb_dropout = 0.1,
    mode = setting.mode
).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=setting.LR, weight_decay=setting.weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=setting.EPOCH//10, gamma=setting.gamma)

In [14]:
best_acc=0
best_epoch=0

global_steps=0

In [15]:
filename=os.path.join(setting.train_result_dir,'traintime_'+time_str+'.txt')
print(filename)

result=open(filename,'a')
start=time.time()
for epoch in range(setting.EPOCH):  # loop over the dataset multiple times
    print('epoch:',epoch)
    result.write('epoch:'+str(epoch)+'\n')
    correct = 0
    total = 0
    
    running_loss = 0.0
    train_loss= 0.0
    net.train()
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        b_inputs=Variable(inputs).to(device)
        b_labels=Variable(labels).to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
#         print(inputs.shape)

        # forward + backward + optimize
        outputs = net(b_inputs)
#         print(labels.unique)
#         print(outputs.unique)
        loss = criterion(outputs, b_labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        global_steps+=1
#         loss_list.append(loss)
            
        # print statistics
        running_loss += loss.item()
        train_loss += loss.item()
        if (global_steps) % 200 == 0:    # print every 2000 mini-batches
            print('global steps %5d running loss: %.3f' %
                  (global_steps, running_loss / 200))
#             state = {
#                 'epoch': epoch,
#                 'step': i,
#                 'net': net.state_dict(),
#                 'optimizer':optimizer.state_dict(),
#                 'loss':running_loss / 200
#             }
#             checkpoint_name=os.path.join(setting.checkpoint_dir,'model_time_'+time_str+'_epoch_'+str(epoch)+'_step_'+str(i)+'.pth')
#             torch.save(state, checkpoint_name)
            running_loss = 0.0
            
        #save loss log file
#         if global_steps%10 ==0:
#             writer.add_scalar('train_loss',loss,global_steps)
            
        
    
    accuracy = 100 * correct / total
    print( 'train oa:', 100 * correct / total,'train loss',train_loss/len(trainloader))
    result.write('train accuracy:'+str(100 * correct / total) +'\t'+'train loss:'+str(train_loss/len(trainloader)))
    #save training accuracy log file
    writer.add_scalar('train_accuracy',accuracy,epoch)
    writer.add_scalar('train_loss',train_loss/len(trainloader),epoch)
    
    
    val_correct = 0.0
    val_total = 0.0
    val_loss = 0.0
    net.eval()
    for valdata in valloader:
        val_inputs, val_labels = valdata
        val_inputs = val_inputs.to(device)
        val_labels =  val_labels.to(device)
        val_outputs = net(val_inputs)
        valloss = criterion(val_outputs, val_labels)
        val_loss += valloss.item()
        _, val_predicted = torch.max(val_outputs.data, 1)
        val_total += val_labels.size(0)
        val_correct += (val_predicted == val_labels).sum().item()
    val_oa=val_correct/val_total
    print(' val oa:',val_oa, 'val_loss:',val_loss/len(valloader))
    result.write('\t val oa:'+str(val_oa)+'\t'+ 'val_loss:'+str(val_loss/len(valloader)))
    writer.add_scalar('val_Accu',val_oa , epoch)
    writer.add_scalar('val_loss',val_loss/len(valloader) , epoch)
    
    if val_oa > best_acc:
        state = {
            'epoch': epoch,
            'accuracy': val_oa,
            'net': net.state_dict(),
            'optimizer':optimizer.state_dict(),
            'loss':loss
        }
        best_model_name=os.path.join(setting.best_model_dir,'model_time_'+time_str+'.pth')
        torch.save(state, best_model_name)
        best_acc=val_oa
        best_epoch=epoch
        print('epoch:',epoch,'best val accuracy:',val_oa)
    
    #pridict  testing data and save accuracy log file
#     if (epoch) % 10 == 0:
#         test_correct = 0.0
#         test_total = 0.0
#         test_loss = 0.0
#         net.eval()
#         for testdata in testloader:
#             test_inputs, test_labels = testdata
#             test_inputs = test_inputs.to(device)
#             test_labels =  test_labels.to(device)
#             test_outputs = net(test_inputs)
#             tsloss = criterion(test_outputs, test_labels)
#             test_loss += tsloss.item()
#             _, test_predicted = torch.max(test_outputs.data, 1)
#             test_total += test_labels.size(0)
#             test_correct += (test_predicted == test_labels).sum().item()
#         oa=test_correct/test_total
#         print(' test oa:',oa, 'loss:',test_loss/len(testloader))
#         result.write('\t test oa:'+str(oa)+'\t'+ 'loss:'+str(test_loss/len(testloader)))
#         writer.add_scalar('Test_Accu',oa , epoch)
#         writer.add_scalar('Test_loss',test_loss/len(testloader) , epoch)
    
#         if oa > best_acc:
#             state = {
#                 'epoch': epoch,
#                 'accuracy': oa,
#                 'net': net.state_dict(),
#                 'optimizer':optimizer.state_dict(),
#                 'loss':loss
#             }
#             best_model_name=os.path.join(setting.best_model_dir,'model_time_'+time_str+'.pth')
#             torch.save(state, best_model_name)
#             best_acc=oa
#             print('epoch:',epoch,'best test accuracy:',oa)
        

    finish_state = {
        'epoch': epoch,
        'net': net.state_dict(),
        'optimizer':optimizer.state_dict(),
        'loss':loss,
        'train_oa':accuracy,
#         'test_oa':oa
        }
end=time.time()
model_name=os.path.join(setting.model_dir,'model_time_'+time_str+'.pth')
torch.save(finish_state, model_name)
#plt.ioff()
#plt.show()
print('Finished Training')
writer.close()
result.close()

./ViT_train_result/traintime_2022_10_31_17_16.txt
epoch: 0
train oa: 51.0 train loss 0.7098739804003865
 val oa: 0.5 val_loss: 0.6939834429799513
epoch: 0 best val accuracy: 0.5
epoch: 1
train oa: 50.9 train loss 0.6993595417193874
 val oa: 0.5486 val_loss: 0.6912001627438752
epoch: 1 best val accuracy: 0.5486
epoch: 2
train oa: 51.85 train loss 0.6976823457440992
 val oa: 0.5 val_loss: 0.7050806393917176
epoch: 3
train oa: 51.45 train loss 0.6978322761420901
 val oa: 0.5882 val_loss: 0.6893843802628564
epoch: 3 best val accuracy: 0.5882
epoch: 4
train oa: 51.6 train loss 0.6957683074257136
 val oa: 0.5176 val_loss: 0.6864967351942561
epoch: 5
train oa: 53.9 train loss 0.6854799063327356
 val oa: 0.5754 val_loss: 0.6736702481452409
epoch: 6
global steps   200 running loss: 0.027
train oa: 57.25 train loss 0.6636881410686728
 val oa: 0.6328 val_loss: 0.6683310978339421
epoch: 6 best val accuracy: 0.6328
epoch: 7
train oa: 59.95 train loss 0.6501521406862032
 val oa: 0.584 val_loss: 0.64

train oa: 78.25 train loss 0.4102133837076668
 val oa: 0.7552 val_loss: 0.4716881117823161
epoch: 73
train oa: 78.8 train loss 0.4117710307553159
 val oa: 0.79 val_loss: 0.4148137883066518
epoch: 74
global steps  2400 running loss: 0.067
train oa: 77.65 train loss 0.4207701882939345
 val oa: 0.7926 val_loss: 0.4145647618354425
epoch: 75
train oa: 79.55 train loss 0.4045874962736571
 val oa: 0.7818 val_loss: 0.4350871394767743
epoch: 76
train oa: 78.15 train loss 0.39919630715355314
 val oa: 0.7798 val_loss: 0.458083001474762
epoch: 77
train oa: 79.1 train loss 0.4089002511678486
 val oa: 0.7788 val_loss: 0.4483194359915226
epoch: 78
train oa: 78.0 train loss 0.3978191201632818
 val oa: 0.786 val_loss: 0.45438562437099134
epoch: 79
train oa: 78.3 train loss 0.40563520738037934
 val oa: 0.7924 val_loss: 0.4161284021723073
epoch: 80
train oa: 77.65 train loss 0.41706666192751274
 val oa: 0.8002 val_loss: 0.40943170507566756
epoch: 81
global steps  2600 running loss: 0.015
train oa: 79.5 t

train oa: 81.7 train loss 0.37627457320277496
 val oa: 0.7784 val_loss: 0.4612044317020128
epoch: 148
train oa: 80.45 train loss 0.3826075242231011
 val oa: 0.764 val_loss: 0.4532559242018713
epoch: 149
global steps  4800 running loss: 0.060
train oa: 80.6 train loss 0.37242668769235515
 val oa: 0.803 val_loss: 0.3866770816541727
epoch: 150
train oa: 79.7 train loss 0.38930641091696505
 val oa: 0.808 val_loss: 0.3976387050363821
epoch: 151
train oa: 80.7 train loss 0.375073469871989
 val oa: 0.806 val_loss: 0.3944007724409364
epoch: 152
train oa: 80.55 train loss 0.3870648049168159
 val oa: 0.7908 val_loss: 0.4253283124943161
epoch: 153
train oa: 79.75 train loss 0.3847733648588281
 val oa: 0.8094 val_loss: 0.39322442061963353
epoch: 154
train oa: 80.15 train loss 0.4023309454918677
 val oa: 0.7824 val_loss: 0.43609138788772434
epoch: 155
train oa: 80.75 train loss 0.3737745948328105
 val oa: 0.7996 val_loss: 0.39990087744058267
epoch: 156
global steps  5000 running loss: 0.015
train o

train oa: 81.75 train loss 0.3689412273126322
 val oa: 0.8204 val_loss: 0.37134321606210735
epoch: 222
train oa: 82.1 train loss 0.36783264557479384
 val oa: 0.8266 val_loss: 0.39132373663285036
epoch: 223
train oa: 81.15 train loss 0.3651731204446346
 val oa: 0.8314 val_loss: 0.3781694330944945
epoch: 224
global steps  7200 running loss: 0.057
train oa: 82.45 train loss 0.3547081831224353
 val oa: 0.8124 val_loss: 0.3805874103241806
epoch: 225
train oa: 81.0 train loss 0.37371207903742604
 val oa: 0.8406 val_loss: 0.3671940024735219
epoch: 225 best val accuracy: 0.8406
epoch: 226
train oa: 82.3 train loss 0.35904706318783336
 val oa: 0.8242 val_loss: 0.3765428149084402
epoch: 227
train oa: 82.35 train loss 0.36254643932676983
 val oa: 0.8216 val_loss: 0.3876547334185007
epoch: 228
train oa: 82.25 train loss 0.34625639634842936
 val oa: 0.8216 val_loss: 0.3940154420097124
epoch: 229
train oa: 83.4 train loss 0.35246913190724066
 val oa: 0.8494 val_loss: 0.35404604025321906
epoch: 229 b

 val oa: 0.8486 val_loss: 0.351564415397216
epoch: 296
train oa: 82.85 train loss 0.3375293984930665
 val oa: 0.8416 val_loss: 0.34942459300109974
epoch: 297
train oa: 82.15 train loss 0.3419950289647881
 val oa: 0.8448 val_loss: 0.3473751340564938
epoch: 298
train oa: 83.05 train loss 0.3426377850130757
 val oa: 0.8498 val_loss: 0.3424345761338352
epoch: 299
global steps  9600 running loss: 0.053
train oa: 83.6 train loss 0.3337204475565075
 val oa: 0.8252 val_loss: 0.3637712604963822
epoch: 300
train oa: 82.95 train loss 0.33464414715439617
 val oa: 0.8474 val_loss: 0.3622726489098591
epoch: 301
train oa: 82.9 train loss 0.34088342163125374
 val oa: 0.8298 val_loss: 0.3831547284941999
epoch: 302
train oa: 81.7 train loss 0.34574571609926363
 val oa: 0.839 val_loss: 0.3636380366181213
epoch: 303
train oa: 83.45 train loss 0.34182506167925547
 val oa: 0.8262 val_loss: 0.38876831247871746
epoch: 304
train oa: 83.65 train loss 0.3305635521373022
 val oa: 0.8532 val_loss: 0.34411994225683

...

 val oa: 0.887 val_loss: 0.3553445055163487
epoch: 1644
train oa: 94.65 train loss 0.12497109608618769
 val oa: 0.8946 val_loss: 0.3479262400218842
epoch: 1645
train oa: 93.9 train loss 0.1434460244614183
 val oa: 0.8862 val_loss: 0.3842459593419109
epoch: 1646
train oa: 93.6 train loss 0.1426915272281732
 val oa: 0.8902 val_loss: 0.37349154172042015
epoch: 1647
train oa: 94.55 train loss 0.1285100419174796
 val oa: 0.895 val_loss: 0.3570088791798599
epoch: 1648
train oa: 93.55 train loss 0.14881122275931627
 val oa: 0.9 val_loss: 0.3546733995122475
epoch: 1649
global steps 52800 running loss: 0.020
train oa: 94.7 train loss 0.12414749767269548
 val oa: 0.8968 val_loss: 0.3443338425532585
epoch: 1650
train oa: 93.55 train loss 0.1483681650592202
 val oa: 0.8942 val_loss: 0.3598674462932869
epoch: 1651
train oa: 93.1 train loss 0.14705924818205174
 val oa: 0.9014 val_loss: 0.3346796178205291
epoch: 1652
train oa: 93.4 train loss 0.16660560583006923
 val oa: 0.8966 val_loss: 0.3311513229

 val oa: 0.899 val_loss: 0.34497704329101136
epoch: 1719
train oa: 95.4 train loss 0.10602706586645382
 val oa: 0.8982 val_loss: 0.36204200464523495
epoch: 1720
train oa: 94.5 train loss 0.12578434037072528
 val oa: 0.9004 val_loss: 0.38816557322234585
epoch: 1721
train oa: 94.1 train loss 0.13140440561147185
 val oa: 0.881 val_loss: 0.38115907444164643
epoch: 1722
train oa: 94.45 train loss 0.1401893118172538
 val oa: 0.8978 val_loss: 0.36693493658178106
epoch: 1723
train oa: 94.75 train loss 0.13238762376452695
 val oa: 0.8998 val_loss: 0.33042730850003343
epoch: 1724
global steps 55200 running loss: 0.022
train oa: 93.8 train loss 0.14046968670465523
 val oa: 0.8832 val_loss: 0.40147972309144786
epoch: 1725
train oa: 93.7 train loss 0.14535348920266017
 val oa: 0.8918 val_loss: 0.34837551748394946
epoch: 1726
train oa: 93.55 train loss 0.1461865133094246
 val oa: 0.907 val_loss: 0.324002945189257
epoch: 1727
train oa: 94.7 train loss 0.12639769488336292
 val oa: 0.9024 val_loss: 0.3

train oa: 95.2 train loss 0.1152300014207473
 val oa: 0.9008 val_loss: 0.3544366037463461
epoch: 1794
train oa: 95.5 train loss 0.12386741347500795
 val oa: 0.8944 val_loss: 0.35900645035151674
epoch: 1795
train oa: 95.05 train loss 0.11666358880542892
 val oa: 0.9012 val_loss: 0.3483637480793469
epoch: 1796
train oa: 94.4 train loss 0.12856999580579667
 val oa: 0.9 val_loss: 0.3697187463034445
epoch: 1797
train oa: 94.55 train loss 0.13229160916444782
 val oa: 0.9022 val_loss: 0.3491947441389488
epoch: 1798
train oa: 94.85 train loss 0.12204801771043101
 val oa: 0.8938 val_loss: 0.36781300000682005
epoch: 1799
global steps 57600 running loss: 0.018
train oa: 95.2 train loss 0.11233415264262814
 val oa: 0.9014 val_loss: 0.36803827026778807
epoch: 1800
train oa: 94.7 train loss 0.12119622726747444
 val oa: 0.8918 val_loss: 0.3747498491025961
epoch: 1801
train oa: 94.55 train loss 0.12574584587308885
 val oa: 0.8958 val_loss: 0.3735823206376534
epoch: 1802
train oa: 94.75 train loss 0.11

train oa: 95.0 train loss 0.11389622594803989
 val oa: 0.9024 val_loss: 0.3388646758667264
epoch: 1869
train oa: 95.05 train loss 0.12184343761778585
 val oa: 0.8936 val_loss: 0.3681813528724456
epoch: 1870
train oa: 93.95 train loss 0.13716658191125436
 val oa: 0.8972 val_loss: 0.36642341315531435
epoch: 1871
train oa: 95.35 train loss 0.10090719812495226
 val oa: 0.9014 val_loss: 0.3523862588759958
epoch: 1872
train oa: 95.15 train loss 0.11328813567286744
 val oa: 0.9052 val_loss: 0.35674794620146233
epoch: 1873
train oa: 94.95 train loss 0.12790238599073236
 val oa: 0.8888 val_loss: 0.40414504569638876
epoch: 1874
global steps 60000 running loss: 0.021
train oa: 94.8 train loss 0.131543813359452
 val oa: 0.8836 val_loss: 0.3806395104497818
epoch: 1875
train oa: 94.15 train loss 0.12601461139042852
 val oa: 0.8962 val_loss: 0.36559428767297997
epoch: 1876
train oa: 95.35 train loss 0.11945653842223394
 val oa: 0.9066 val_loss: 0.3296254800781573
epoch: 1877
train oa: 94.8 train loss

train oa: 95.15 train loss 0.10173033639624214
 val oa: 0.8954 val_loss: 0.3828200562228981
epoch: 1944
train oa: 95.9 train loss 0.10176874687268501
 val oa: 0.9002 val_loss: 0.3710275131898534
epoch: 1945
train oa: 94.85 train loss 0.11164517681220701
 val oa: 0.8938 val_loss: 0.39850497969925797
epoch: 1946
train oa: 94.85 train loss 0.10357623537530919
 val oa: 0.8968 val_loss: 0.40358844721391346
epoch: 1947
train oa: 95.5 train loss 0.1092771183089616
 val oa: 0.8944 val_loss: 0.4089328926547962
epoch: 1948
train oa: 95.15 train loss 0.11578529030316671
 val oa: 0.8882 val_loss: 0.43694411128957866
epoch: 1949
global steps 62400 running loss: 0.019
train oa: 95.1 train loss 0.1182779082947679
 val oa: 0.8984 val_loss: 0.3681627382229102
epoch: 1950
train oa: 95.8 train loss 0.09875902003172476
 val oa: 0.8912 val_loss: 0.41943648319859245
epoch: 1951
train oa: 96.0 train loss 0.10147474169260746
 val oa: 0.885 val_loss: 0.4123640891570701
epoch: 1952
train oa: 95.55 train loss 0.

In [16]:
print('best_acc:',best_acc)
print('best_epoch:',best_epoch)
print('train time:', end-start)
print(best_model_name)
print(model_name)

best_acc: 0.9108
best_epoch: 1748
train time: 14005.140038013458
./model/model_best/model_time_2022_10_31_17_16.pth
./model/model_time_2022_10_31_17_16.pth


In [17]:

# model_name='./model/salinas/model_best/model_time_2021_03_18_12_17.pth'
# model_name='.\model\salinas\model_time_2020_03_31_11_43.pth'
# model = torch.load(model_name)
model = torch.load(best_model_name)

In [18]:
net.load_state_dict(model['net'])
# optimizer.load_state_dict(model['optimizer'])
# start_epoch = model['epoch'] + 1
# loss=model['loss']

<All keys matched successfully>

In [19]:
predict_trainlabels=[]
trainlabels=[]
t1=time.time()
with torch.no_grad():
    for trdata in trainloader:
        trinputs, trlabels = trdata
        trinputs = trinputs.to(device)
        trlabels = trlabels.to(device)
        troutputs = net(trinputs)
        _, trpredicted = torch.max(F.softmax(troutputs), 1)
        predict_trainlabels.extend(trpredicted)
        trainlabels.extend(trlabels)
    print('predict training set finished')
#print(len(trainlabels))
#print(len(predict_trainlabels))
t2=time.time()
print('predict train time:',t2-t1)
oa_train, aa_train, kappa_train, acc_train=cf.eval_results_own(predict_trainlabels,trainlabels,2)
print('OA_train:',oa_train, '\nAA_train:', aa_train, '\nkappa_train:', kappa_train, '\nacc_train:', acc_train)

/home/user/anaconda3/envs/lfs-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


predict training set finished
predict train time: 1.3504996299743652
OA_train: 0.966 
AA_train: 0.966 
kappa_train: 0.9319999999999999 
acc_train: [0.954 0.978]


In [20]:
predict_vallabels=[]
val_labels=[]
t1=time.time()
with torch.no_grad():
    for valdata in valloader:
        vinputs, vlabels = valdata
        vinputs = vinputs.to(device)
        vlabels = vlabels.to(device)
        voutputs = net(vinputs)
        _, vpredicted = torch.max(F.softmax(voutputs), 1)
        predict_vallabels.extend(vpredicted)
        val_labels.extend(vlabels)
    print('predict val set finished')
#print(len(trainlabels))
#print(len(predict_trainlabels))
t2=time.time()
print('predict val time:',t2-t1)
oa_val, aa_val, kappa_val, acc_val=cf.eval_results_own(predict_vallabels,val_labels,2)
print('OA_val:',oa_val, '\nAA_val:', aa_val, '\nkappa_val:', kappa_val, '\nacc_val:', acc_val)

/home/user/anaconda3/envs/lfs-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


predict val set finished
predict val time: 3.4086055755615234
OA_val: 0.9108 
AA_val: 0.9108 
kappa_val: 0.8216000000000001 
acc_val: [0.8892 0.9324]
